In [5]:
import numpy as np
from matplotlib import pyplot as plt
import cv2 as cv
import math
import os

In [6]:
#convinience function
path_models = os.path.join("product-recognition-on-store-shelves-images","object_detection_project","models")
path_scenes = os.path.join("product-recognition-on-store-shelves-images","object_detection_project","scenes")

def get_path_model(index):
    return os.path.join(path_models, f"{index}.jpg")

def get_path_scene(index):
    return os.path.join(path_scenes, f"{index}")

def rotate_tuple(point, angle_degrees):
    """
    Rotates a 2D tuple (point) by a given angle in degrees.

    Args:
        point: A tuple (x, y).
        angle_degrees: Angle to rotate the point, in degrees.

    Returns:
        A tuple (x', y') representing the rotated point.
    """
    # Convert angle to radians
    angle_radians = math.radians(angle_degrees)

    # Extract the x and y coordinates
    x, y = point

    # Apply the rotation matrix
    x_rotated = x * math.cos(angle_radians) - y * math.sin(angle_radians)
    y_rotated = x * math.sin(angle_radians) + y * math.cos(angle_radians)

    return (x_rotated, y_rotated)

def multiply_tuple(point, scale):
    return tuple(element * scale for element in point)

def add_tuple(x, y):
    return (x[0] + y[0], x[1] + y[1])

In [7]:
def find_image(scene_path, model_paths, draw_on_color=True):
    target_resolution = (150, 210)
    scene_gray = cv.imread(scene_path,cv.IMREAD_GRAYSCALE)

    scene_to_draw_on = cv.imread(scene_path) if draw_on_color else scene_gray

    #create array with all models
    models = []
    for model_gray_path in model_paths:

        name = os.path.split(model_gray_path)[-1]
        models.append({
            "name": name,
            "img": cv.imread(model_gray_path, cv.IMREAD_GRAYSCALE)
            })

    for model in models:
        print(f"Looking for {model["name"]}...")

        model_gray = model["img"]
        #_model_gray = model["img"]
        #model_gray = cv.resize(_model_gray, target_resolution, interpolation=cv.INTER_AREA)
        MIN_MATCH_COUNT = 30

        # Initiate SIFT detector
        sift = cv.SIFT_create()

        # find the keypoints and descriptors with SIFT
        kp1, des1 = sift.detectAndCompute(model_gray,None)
        kp2, des2 = sift.detectAndCompute(scene_gray,None)

        FLANN_INDEX_KDTREE = 1
        index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
        search_params = dict(checks = 50)
        flann = cv.FlannBasedMatcher(index_params, search_params)
        matches = flann.knnMatch(des1,des2,k=2)

        # store all the good matches as per Lowe's ratio test.
        good = []
        for m,n in matches:
            if m.distance < 0.7 * n.distance:
                good.append(m)

        scene_to_draw_on = scene_to_draw_on if scene_to_draw_on is not None else scene_gray

        result = scene_to_draw_on

        if len(good)>MIN_MATCH_COUNT:
            print( "Enough matches are found - {}/{}".format(len(good), MIN_MATCH_COUNT) )
            src_pts = [ kp1[m.queryIdx] for m in good ]
            dst_pts = [ kp2[m.trainIdx] for m in good ]

            #bary_src = sum(key_pt.pt for key_pt in src_pts)/len(src_pts)
            #print(bary_src)

            bary_src = (0,0)
            for key_pt in src_pts:
                bary_src = (bary_src[0] + key_pt.pt[0], bary_src[1] + key_pt.pt[1])
            bary_src = (bary_src[0]/len(src_pts), bary_src[1]/len(src_pts))
            print(bary_src)

            #img_visualization = cv.drawKeypoints(model_gray,src_pts,None,flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
            #plt.imshow(img_visualization)
            #plt.show()

            joint_vec_src = []
            for key_pt in src_pts:
                vec = (key_pt.pt[0] - bary_src[0], key_pt.pt[1] - bary_src[1])
                joint_vec_src.append(vec)

            bary_vote_dst = []
            for i in range(len(dst_pts)):
                rot = dst_pts[i].angle - src_pts[i].angle
                scale = dst_pts[i].size / src_pts[i].size
                bary = add_tuple(src_pts[i].pt, multiply_tuple(rotate_tuple(joint_vec_src[i], rot), scale))
                bary_vote_dst.append(bary)

            for point in bary_vote_dst:
                # Draw a circle at each coordinate
                cv.circle(scene_gray, (int(point[0]), int(point[1])), radius=5, color=(0, 255, 0), thickness=-1)  # Green filled circle

            # Show the image
            cv.imshow("Image with Points", scene_gray)
            cv.waitKey(0)
            cv.destroyAllWindows()
            """ USE THIS TO DRAW THE RESULTS FROM POINTS
            src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
            dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
            M, mask = cv.findHomography(src_pts, dst_pts, cv.RANSAC,5.0)
            matchesMask = mask.ravel().tolist()
            h,w = model_gray.shape
            pts = np.float32([ [0,0],[0,h-1],[w-1,
                                            h-1],[w-1,0] ]).reshape(-1,1,2)

            dst = cv.perspectiveTransform(pts,M)

            #add square and add text
            means = np.mean(dst, axis=0)
            x, y = [a for a in means[0]]
            x = x * 0.8 #adjust for text

            result = cv.polylines(scene_to_draw_on,[np.int32(dst)],True,255,3, cv.LINE_AA)

            boldness = 3
            size = 3
            cv.putText(scene_to_draw_on, model["name"], (int(x),int(y)), cv.FONT_HERSHEY_COMPLEX_SMALL, size, (0, 0, 255, 255), boldness, cv.LINE_AA)
            """
        else:
            print( "Not enough matches are found - {}/{}".format(len(good), MIN_MATCH_COUNT) )
            matchesMask = None

    print("------------------------------------------")
    print("Results:")
    return result


#define models to be searched in image
models = [get_path_model(a) for a in [0]]#[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]]
#[0,1,11,19,24,25,26]]
result = find_image(scene_path=get_path_scene("e1.png"),
                    model_paths=models,
                    draw_on_color=True
                    )

plt.imshow(cv.cvtColor(result, cv.COLOR_BGR2RGB))
plt.show()

Looking for 0.jpg...
Enough matches are found - 464/30
(581.5304604275473, 726.7179503934137)


KeyboardInterrupt: 

In [33]:
import cv2
import numpy as np

# Create a blank image (optional, if you already have an image, use that)
image = np.zeros((500, 500, 3), dtype=np.uint8)  # Black image of size 500x500

# List of tuple coordinates
coordinates = [(100, 100), (200, 200), (300, 300), (400, 400)]

# Iterate through the list and plot each coordinate
for point in coordinates:
    # Draw a circle at each coordinate
    cv2.circle(image, point, radius=5, color=(0, 255, 0), thickness=-1)  # Green filled circle

# Show the image
cv2.imshow("Image with Points", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

2025-01-20 17:58:17.027 python[1351:22732] +[IMKClient subclass]: chose IMKClient_Modern
2025-01-20 17:58:17.027 python[1351:22732] +[IMKInputSession subclass]: chose IMKInputSession_Modern
